## Scopes

This will be a very brief intro on scopes.  Python, unlike most modern languages is not lexically scoped.

> What does `lexically scoped` mean?  Lexical means essentially, words, but it has to do with "boundaries" of your code.  
> In many languages with curly braces, the braces define a scope.  Variables or other symbols introduced in that scope  
> (eg the curly braces) only last inside the brace section.

Python is not like that.  

- Indentation does **not** create a new scope in python
- a scope does however _bind_ a `symbol`. 

So we need to discuss what symbols are, and how they are mapped in `namespaces`.  In short:

```
scopes -> namespaces -> symbols
```

## Namespaces and symbols

Many people who program tend to forget or never truly learn what a `symbol` in a language is.  For example consider this
one-liner:

```python
path = "/tmp/some-data"
```

As programmers, we think in our heads:

```
path is a variable that has the value '/tmp/some-data'`
```

But that's wrong in several ways.  More precisely:

- `path` is a symbol...
- that is a look up into a table...
- where the key is the name `path`...
- and the value is a reference to an object.  
- The object that symbol points to:
    - has a memory location 
    - and a type.  
- The type of the object determines how we get the actual value it refers to

## Namespace as a dict

Conceptually, you can think of a namespace like a dictionary that python looks up to find symbols, so that when you  
make use of a symbol, python knows how to retrieve the value based on the symbol name.

```yaml
global-main: # This is the scope which is global (or module) level
  path:      # this is the symbol name
    id: 0x7f14297171b0,  # this is the object's memory location
    type: str            # and this is the type
```

But, there's a potential problem.  What if we have two symbols with the same name, but in different parts of the code?

In [ ]:
from pprint import pprint

lookup_val = 10  # Global scoped.  Global is actually module-scoped (only builtins are truly global)

def lookup():
    lookup_val = 100  # local scoped (to )
    print(f"lookup(): {locals()}")
    #pprint(globals())

    def inner():
        print(f"in inner(): lookup_val = {lookup_val}")
        #print(f"inner(): {locals()}")

    print(f"In lookup: lookup_val = {lookup_val}")
    return inner

inner = lookup()
inner()  # which lookup_val is inner going to use?
print(f"In global scope: lookup_val is {lookup_val}") # it's still 10

In [ ]:
globals()

In [ ]:
a_list = [1, 2]

# Because a_list is a mutable reference and it's in the global (top-level) scope, the call here will mutate a_list
def captured_a_list(val: int):
    a_list.append(val)
    # print(locals())

captured_a_list(10)

In [ ]:
print(a_list)
def replacer(some_list: list[int]):
    print(f"id of some_list = {id(some_list)}")
    some_list = []
    print(f"id of some_list after assignment is now = {id(some_list)}")
    print(f"but id of a_list is still = {id(a_list)}")

print(f"id of a_list is {id(a_list)}")
replacer(a_list) # this is like lookup(), it creates a new binding, leaving the original alone
print(a_list)

## Python scope lookup

There are 4 kinds of scope in python in this order of lookup

- local
- enclosing
- global
- builtin

```
+-----------------------------------+
| builtins                          |
|                                   |
| +-------------------------------+ |
| | globals                       | |
| |                               | |
| | +---------------------------+ | |
| | | enclosing/non-locals      | | |
| | |                           | | |
| | | +----------------------+  | | |
| | | | locals               |  | | |
| | | +----------------------+  | | |
| | +---------------------------+ | |
| +-------------------------------+ |
+-----------------------------------+
```

You might wonder what is the difference between local and enclosing?

## The global and nonlocal keywords

Though rarely used, sometimes it is necessary to mark a variable in a non top-level scope (ie, declared inside of a 
function, method, or class) as being either `global` or `nonlocal`.

Let's look at an example:

In [ ]:
def globalizer():
    global foo
    foo = "testing"

globalizer()
# Where did `foo` come from?  It got inserted into the global namespace through the `global` keyword
print(f"foo = {foo}")

def change_foo():
    foo = "hello"  # What do you think this will print?

## What will change_foo print?

In [ ]:
change_foo()
print(f"foo did not change and is still = {foo}")

# So how DO you change it then?

### A mutable global

As we can see, we tried to reassign the `global` variable foo inside of `change_foo` function, but it didn't work.  Is
there a way to make it work?

There is, but you have to add the `global` keyword to the function that will mutate the global variable.

In [ ]:
name = "sean"  # A global (aka module scoped) variable since it is not inside a function or class

def non_global_rename(to: str):
    name = to  # Question: what scope does `name` live in?

non_global_rename("john")
print(f"without `global name`, name is still = {name}")

def rename(to: str):
    global name
    name = to

print(f"Before call to rename, name = {name}")
rename("john") # because we use global, we reassign it
print(f"after call to rename, name is now = {name}")

In [ ]:
def change_foo():
    global foo
    foo = "hello"  # What do you think this will print?

change_foo()
print(f"foo is now {foo}")

## Mutable variables and scope

But wait! you say.  I have written code like this that can change an outer variable.

In [ ]:
from datetime import datetime


build_to_date = {}

def add_to_map(key: int):
    build_to_date[key] = datetime.now()
    return build_to_date

In [ ]:
import time


for i in range(3):
    print(add_to_map(i))
    time.sleep(1)

## What's going on??

Notice, that when we tried to mutate `foo` which was a `str`, we couldn't do it and had to add the `global` keyword.
So how did this work with a dict?  That's because in python a str type is immutable.  There's not many immutable types
in python, and generally speaking, unless it's a str, int, tuple or something extending those classes, it's mutable.
More importantly, it's a mutable _reference_.

This means you are _directly_ changing the value of the argument, rather than passing in a _copy_.  In some languages,
they call this _pass by reference_ as opposed to _pass by copy_.

## More scoping gotchas

This is a common source of confusion for new pythonistas
score = 50 # try setting this to 50

In [ ]:
def grader(check: int):
    # dont have to create a "default" grade here,  grade is scoped to the entire function
    # however, try changing score to 50, and commenting out the marked lines
    if check > 90:
        grade = "A"
    elif check > 80:
        grade = "B"
    elif check > 70:
        grade = "C"
    elif check > 60:
        grade = "D"
    #else:              # try commenting this out
    #    grade = "F"    # and this
    return grade


grader(score)

## Higher order functions

Part of the reason that namespaces and scopes were covered, is to help better understand how higher order functions
work.  In python, functions are:

- first class citizens (unlike Java or C/C++) 
- can be used as arguments to functions or returned from functions
- capture their namespace, namely, their local and enclosing scoped namespaces
    - In a sense, you can think of functions as mini-objects that contain state.

We will next cover:

- nested functions
- functions as return values
- functions as arguments

In [ ]:
# A common use case of a nested function, is to create a "private" function that handles some functionality

import functools as ft
from random import randint
from typing import Callable, Generator, Sequence

# Example of a function that returns a function
# Question: what scope is `num` in? (hint: from whose perspective?)
def die_pool(num: int):
    if num < 1:
        raise Exception(f"num must be greater than 0")
    def inner(size: int):
        if size < 2:
            raise Exception("size must be >= 2")
        if size > 1000:
            raise Exception("Only up to d1000 is possible")
        # Returns a generator, which I will cover when we go over the Iterator protocol.  Think of it as a lazy list
        return (randint(1, size) for _ in range(num))
    return inner

# Example of a nested function
def die_roll_v1(
    num: int, 
    size: int, 
    target: int
):
    """Rolls num of size dice.  Returns a 2 element tuple of rolls < target, and >= target

    Parameters
    ----------
    num : int
        number of dice to roll
    size : int
        size of the die
    target : int
        A number from 1 - size
    """
    if target < 1 or target > size:
        raise Exception("target must be: 1 <= target <= size")

    # Example of a nested function.  This is one way of making something "private" in python.  Technically, you can 
    # still access 
    def filter(acc: tuple[list[int], list[int]], next: int):
        if next >= target:
            acc[1].append(next)
        else:
            acc[0].append(next)
        return acc
    
    roll = die_pool(num)(size)
    result = ft.reduce(filter, roll, ([], []))
    return sorted(result[0]), sorted(result[1])

In [ ]:
failed, succeeded = die_roll_v1(6, 20, 12)
print(failed, succeeded)

In [ ]:
# This is an example of a function as a parameter.  Here, we are passing in `roll`, which is a function that takes no
# arguments, and returns a Generator.  I will go more in depth in Generators when we cover the Iterator protocol.  For
# now, think of it as a lazy list that yields one element at a time
def die_roll_v2(
    pool: Callable[[], Generator[int, None, None]], 
    target: int
):
    """Takes a roll of dice (list of int) and finds all values >= to target.  If exploding is not None, if any value
    is >= to exploding, roll another die (recursively)

    []

    Parameters
    ----------
    roll : list[int]
        _description_
    target : int
        _description_
    """

    def filter(acc: tuple[list[int], list[int]], next: int):
        if next >= target:
            acc[1].append(next)
        else:
            acc[0].append(next)
        return acc
    
    roll = pool()
    result = ft.reduce(filter, roll, ([], []))
    return sorted(result[0]), sorted(result[1])

In [ ]:
p5d20 = lambda: die_pool(5)(20)
p7d20 = lambda: die_pool(7)(20)
failed, successes = die_roll_v2(p7d20, 12)
print(failed, successes)

In [ ]:
# Example of a stateful closure.  And this is one reason I spent all that time going over scopes and namespaces

def find(match: str):
    count = 0

    def search(text: str):
        nonlocal count  # Try commenting this out
        if match in text:
            count += 1
        return count

    return search

matcher = find("sean")
text = [
    "Hi sean how are you?",
    "Did sean do his work today?",
    "Why not?",
    "Excuses excuses",
    "Get cracking sean!"
]
for line in text:
    count = matcher(line)
    print(f"found {count} matches so far")
